In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load credentials from .env file
load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

# Create database connection URL (for MySQL; change driver if needed)
db_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create SQLAlchemy engine
engine = create_engine(db_url)

In [ ]:
# Step 1: Load tables
abpusers = pd.read_sql("SELECT Id, Name, ManagerId FROM abpusers", engine)
abpuserroles = pd.read_sql("SELECT UserId, RoleId FROM abpuserroles", engine)
abproles = pd.read_sql("SELECT Id, Name FROM abproles", engine)

In [ ]:
# Step 2: Officer + officer_role
officer_df = abpusers.merge(
    abpuserroles,
    left_on="Id",
    right_on="UserId",
    how="inner"
).merge(
    abproles.add_prefix("officer_"),
    left_on="RoleId",
    right_on="officer_Id",
    how="inner"
)

In [ ]:
# Step 3: Supervisor + supervisor_role
supervisor_df = abpusers.merge(
    abpuserroles,
    left_on="Id",
    right_on="UserId",
    how="inner"
).merge(
    abproles.add_prefix("supervisor_"),
    left_on="RoleId",
    right_on="supervisor_Id",
    how="inner"
)

In [ ]:
# Step 4: Join officers with their supervisors
df = officer_df.merge(
    supervisor_df,
    left_on="ManagerId",
    right_on="Id",
    suffixes=("_officer", "_supervisor")
)

In [7]:
# Step 5: Select + rename columns
final_df = df[[
    "Name_supervisor",          # Supervisor name
    "supervisor_Name",          # Supervisor role name
    "Name_officer",             # Officer name
    "officer_Name"              # Officer role name
]].rename(columns={
    "Name_supervisor": "SupervisorName",
    "supervisor_Name": "SupervisorRole",
    "Name_officer": "RelationshipOfficerName",
    "officer_Name": "OfficerRole"
}).reset_index(drop=True)
final_df

,SupervisorName,SupervisorRole,RelationshipOfficerName,OfficerRole
0,Manager1,PiccTaskManager,Investigator2,PiccTaskInvestigator
1,Manager2,PiccTaskManager,Contractor1,PiccTaskContractor
2,Manager2,PiccTaskManager,Contractor2,PiccTaskContractor
3,admin,PiccTaskManager,InvestigatorGiza,PiccTaskInvestigator
4,admin,PiccTaskManager,ContractorGiza,PiccTaskContractor
5,LMAO,PiccTaskManager,MAD,PiccTaskInvestigator
6,InvestigatorMAADIManager,PiccTaskManager,InvestigatorMAADI,PiccTaskInvestigator
7,ContarctorHelio,PiccTaskManager,ContractorHelio1,PiccTaskContractor
8,ServManager,PiccTaskManager,ServInvest1,PiccTaskInvestigator
9,ServManager,PiccTaskManager,ServInvest2,PiccTaskInvestigator
